In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-06-09 23:11:28--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8001::154, 2606:50c0:8002::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.08s   

2024-06-09 23:11:28 (14.1 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read() 

In [3]:
print("length of the dataset ", len(text))

length of the dataset  1115394


In [4]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [5]:

chars = sorted(list(set(text))) 
vocab_size = len(chars)
print("chars:", ''.join(chars))
print(f"{vocab_size=}")

chars: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab_size=65


In [6]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[n] for n in l])
print(encode("hi there"))
print(decode(encode("hi there")))

[46, 47, 1, 58, 46, 43, 56, 43]
hi there


In [7]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(f"{data.shape=} {data.dtype=}")
print(f"{data[:100]=}")

data.shape=torch.Size([1115394]) data.dtype=torch.int64
data[:100]=tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [8]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [9]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when context is {context} target is {target}")


when context is tensor([18]) target is 47
when context is tensor([18, 47]) target is 56
when context is tensor([18, 47, 56]) target is 57
when context is tensor([18, 47, 56, 57]) target is 58
when context is tensor([18, 47, 56, 57, 58]) target is 1
when context is tensor([18, 47, 56, 57, 58,  1]) target is 15
when context is tensor([18, 47, 56, 57, 58,  1, 15]) target is 47
when context is tensor([18, 47, 56, 57, 58,  1, 15, 47]) target is 58


In [13]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    print(f"{ix=}")
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+1+block_size] for i in ix])
    return x, y
xb, yb = get_batch('train')
# print(f"{xb=}\n{yb=}")
print("inputs:")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)
print(yb)
print('-----')

for b in range(batch_size): # batch dimesion
    for t in range(block_size): # time dimesion
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f'when context is {context.tolist()} the target is {target}')


ix=tensor([ 76049, 234249, 934904, 560986])
inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
-----
when context is [24] the target is 43
when context is [24, 43] the target is 58
when context is [24, 43, 58] the target is 5
when context is [24, 43, 58, 5] the target is 57
when context is [24, 43, 58, 5, 57] the target is 1
when context is [24, 43, 58, 5, 57, 1] the target is 46
when context is [24, 43, 58, 5, 57, 1, 46] the target is 43
when context is [24, 43, 58, 5, 57, 1, 46, 43] the target is 39
when context is [44] the target is 53
when context is [44, 53] the target is 56
when context is [44, 53, 56] the target is 1
when context is [44, 53, 5